In [1]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-storage firebase-admin
!pip install langchain_community
!pip install google-cloud-aiplatform
!pip install google-cloud-storage
!pip install langchain_community
!pip install langchain
!pip install pypdf

INFO: pip is looking at multiple versions of google-api-python-client to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 22.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 58.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 kB 22.3 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are ins

In [2]:
import IPython
from IPython.display import Markdown, display
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")


import vertexai

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("fpc-manual.pdf")
pages = loader.load_and_split()
print(len(pages))
print(pages[10])

126
page_content='/H20321/H20321Store foods away from dripping\ncondensate , at least six inches above\nthe floor and with enough space\nbetween items to encourage aircirculation.\nFreezer Storage\nFreezing is an excellent method\nfor prolonging the shelf life of foods.By keeping foods frozen solid, thebacterial growth is minimal at best.However, if frozen foods are thawedand then refrozen, then harmfulbacteria can reproduce to dangerouslevels when thawed for the secondtime. In addition to that, the quality ofthe food is also affected. Never re-freeze thawed foods, instead use themimmediately. Keep the followingrules in mind for freezer storage:\n/H20321/H20321Use First In First Out method of\nstock rotation.\n/H20321/H20321All frozen foods should be frozen\nsolid with temperature at 0°F orlower.\n/H20321/H20321Always use clean containers that\nare clearly labeled and marked,and have proper and secure lids. \n/H20321/H20321Allow adequate spacing between\nfood containers to allow for pr

In [5]:
import vertexai
from vertexai.language_models import TextEmbeddingModel
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@002")
# Generate embeddings for each page
embeddings = embeddings = [embedding_model.get_embeddings([page.page_content])[0] for page in pages]

In [6]:
# Extract the vector from the first embedding to verify
vector = embeddings[0].values
print(f"Length of the first vector: {len(vector)}")
print(f"First vector: {vector}")

Length of the first vector: 768
First vector: [-0.014551563188433647, -0.0347423329949379, -0.05348101630806923, 0.023631349205970764, 0.007029604632407427, -0.052199747413396835, 0.022074319422245026, 0.02028043009340763, 0.011824054643511772, 0.04181516170501709, -0.008813895285129547, -0.012041384354233742, 0.05240508168935776, 0.007099385838955641, -0.02535090409219265, -0.008589171804487705, -0.062673419713974, -0.026467768475413322, 0.047733016312122345, 0.0324660986661911, -0.06832766532897949, -0.04218829795718193, 0.006558739580214024, 0.02518017403781414, 0.0004946303670294583, -0.08242819458246231, -0.02580430544912815, 0.017687266692519188, -0.014445845037698746, 0.029930753633379936, -0.04174208268523216, 0.0033266975078731775, -0.03375258669257164, 0.013336879201233387, 0.020129090175032616, 0.010927137918770313, -0.016849493607878685, 0.032285176217556, 0.012363732792437077, 0.06759518384933472, 0.013745409436523914, -0.0233285054564476, 0.04777534678578377, 0.0298375040

In [7]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use the application default credentials.
cred = credentials.ApplicationDefault()

firebase_admin.initialize_app(cred)
db = firestore.client()

In [8]:
# Store the text of each page in Firestore
collection_name = "page_content"
for i, page in enumerate(pages):
    doc_id = f"page_{i+1}"
    content = page.page_content
    db.collection(collection_name).document(doc_id).set({"content": content})

print("All pages have been stored in Firestore.")

All pages have been stored in Firestore.


In [10]:
import json

with open("embeddings.json", "w") as f:
  for i, embedding in enumerate(embeddings):
    
    embedding_list = embedding.values
    data = {"id": str(i + 1), "embedding": embedding_list}  # Cast page number to string
    json.dump(data, f)
    f.write("\n")  # Add newline for JSON-L format

print("Embeddings saved to embeddings.json")


Embeddings saved to embeddings.json


In [11]:
BUCKET_URI = f"gs://{PROJECT_ID}"

In [12]:
! gsutil cp "embeddings.json" "$BUCKET_URI"

E0613 18:09:40.395745223    1666 backup_poller.cc:127]                 Run client channel backup poller: UNKNOWN:pollset_work {created_time:"2024-06-13T18:09:40.395194614+00:00", children:[UNKNOWN:Bad file descriptor {syscall:"epoll_wait", os_error:"Bad file descriptor", errno:9, created_time:"2024-06-13T18:09:40.395069873+00:00"}]}
Copying file://embeddings.json [Content-Type=application/json]...
/ [1 files][  2.0 MiB/  2.0 MiB]                                                
Operation completed over 1 objects/2.0 MiB.                                      


In [13]:
# init the aiplatform package
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [14]:
# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"vs-quickstart-index-{UID}",
    contents_delta_uri=BUCKET_URI,
    dimensions=768,
    approximate_neighbors_count=10,
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/1044046007369/locations/us-central1/indexes/530144924494987264/operations/6835344141237354496
MatchingEngineIndex created. Resource name: projects/1044046007369/locations/us-central1/indexes/530144924494987264
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/1044046007369/locations/us-central1/indexes/530144924494987264')


In [21]:
# create IndexEndpoint
assessment_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"assessment-index-endpoint", public_endpoint_enabled=True
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/1044046007369/locations/us-central1/indexEndpoints/2546350182673416192/operations/4233952406477471744
MatchingEngineIndexEndpoint created. Resource name: projects/1044046007369/locations/us-central1/indexEndpoints/2546350182673416192
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/1044046007369/locations/us-central1/indexEndpoints/2546350182673416192')


In [24]:
DEPLOYED_INDEX_ID = f"vs_quickstart_deployed2_{UID}"

In [ ]:
# deploy the Index to the Index Endpoint
assessment_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/1044046007369/locations/us-central1/indexEndpoints/2546350182673416192
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/1044046007369/locations/us-central1/indexEndpoints/2546350182673416192/operations/3088349251265101824
